In [ ]:
from op_analytics.datasources.platform_metrics.pg_daily_pull import PostgresDailyPull

meta = PostgresDailyPull.fetch()
meta

In [ ]:
from op_analytics.datasources.platform_metrics.prometheus_daily_pull import PrometheusDailyPull

meta_prom = PrometheusDailyPull.fetch()
meta_prom

In [ ]:
from op_analytics.datasources.platform_metrics import execute
import os

# Override to force BQ write.
os.environ["OPLABS_ENV"] = "prod"

result = execute.execute_pull()

In [ ]:
from op_analytics.datasources.platform_metrics import execute
from op_analytics.coreutils.partitioned.dailydatawrite import write_to_prod

with write_to_prod():
    result = execute.execute_pull_1()

In [ ]:
#Backfill tables [Currently troubleshooting]
#from op_analytics.coreutils.partitioned.location import DataLocation

# # Override location for local. Othersise It will default to DataLocation.LOCAL when running from laptop.
#     def mock_location():
#         return DataLocation.GCS

import os

from op_analytics.coreutils.bigquery.write import (
    overwrite_unpartitioned_table,
)
from op_analytics.coreutils.logger import structlog

from op_analytics.datasources.platform_metrics.pg_daily_pull import PostgresDailyPull
from op_analytics.datasources.platform_metrics.prometheus_daily_pull import PrometheusDailyPull
from op_analytics.coreutils.partitioned.dailydatawrite import write_to_prod

import os
from unittest.mock import patch

from op_analytics.coreutils.partitioned import dailydatawrite
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datasources.platform_metrics.dataaccess import PlatformMetrics


with write_to_prod():

    log = structlog.get_logger()

    # BigQuery Dataset and Tables
    BQ_DATASET = "platform_metrics"
    ANALYTICS_TABLE_PG = "platform_metrics_jobs_v1"
    ANALYTICS_TABLE_PROM = "platform_metrics_prometheus_metrics_v1"

    # current_env = OPLabsEnvironment.PROD
    def write_pg_to_bq(data: PostgresDailyPull):
        """Write to BigQuery.

        This operation will be retired soon. Data will move to GCS + Clickhouse.
        """
        jobs_df = data.jobs_df.rename({"dt": "date"})
        # jobs_df = data.jobs_df_truncated.rename({"dt": "date"})
        
        overwrite_unpartitioned_table(
            df=jobs_df,
            dataset=BQ_DATASET,
            table_name=ANALYTICS_TABLE_PG,
        )

        return {"jobs_df": len(jobs_df)}

    def write_prom_to_bq(data: PrometheusDailyPull):
        """Write to BigQuery.

        This operation will be retired soon. Data will move to GCS + Clickhouse.
        """
        metrics_df = data.metrics_df.rename({"dt": "date"})
        # metrics_df = data.metrics_df_truncated.rename({"dt": "date"})

        overwrite_unpartitioned_table(
            df=metrics_df,
            dataset=BQ_DATASET,
            table_name=ANALYTICS_TABLE_PG,
        )

        return {"metrics_df": len(metrics_df)}

    data_pg = PostgresDailyPull.fetch()
    data_prom = PrometheusDailyPull.fetch()

    summary = {}
    summary["bigquery"] = {"pg": write_pg_to_bq(data_pg), "prom": write_prom_to_bq(data_prom)}
    summary

    metrics = PrometheusDailyPull.fetch()
    jobs = PostgresDailyPull.fetch()


    def overwrite_reviews(metrics, jobs):
        """Overwrite GCS with the backfill reviews df."""
        
        # Required to allow writing data to GCS from local.
        os.environ["ALLOW_WRITE"] = "true"

        # Override location for local. Othersise It will default to DataLocation.LOCAL when running from laptop.
        def mock_location():
            return DataLocation.GCS

        with patch.object(dailydatawrite, "determine_location", mock_location):
            PlatformMetrics.JOBS.write(
                dataframe=metrics,
                sort_by=["dt", "id", "pipeline_id", "workflow_id"],
            )

            PlatformMetrics.PROMETHEUS_METRICS.write(
                dataframe=jobs,
                sort_by=["dt", "unix_time"],
            )


    overwrite_reviews(metrics, jobs)

In [ ]:
import os
from unittest.mock import patch

from op_analytics.coreutils.partitioned import dailydatawrite
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datasources.platform_metrics.dataaccess import PlatformMetrics

metrics = PrometheusDailyPull.fetch()
jobs = PostgresDailyPull.fetch()


def overwrite_reviews(metrics, jobs):
    """Overwrite GCS with the backfill reviews df."""
    
    # Required to allow writing data to GCS from local.
    os.environ["ALLOW_WRITE"] = "true"

    # Override location for local. Othersise It will default to DataLocation.LOCAL when running from laptop.
    def mock_location():
        return DataLocation.GCS

    with patch.object(dailydatawrite, "determine_location", mock_location):
        PlatformMetrics.JOBS.write(
            dataframe=metrics,
            sort_by=["dt", "id", "pipeline_id", "workflow_id"],
        )

        PlatformMetrics.PROMETHEUS_METRICS.write(
            dataframe=jobs,
            sort_by=["dt", "unix_time"],
        )


overwrite_reviews(metrics, jobs)